<a href="https://colab.research.google.com/github/park1NG/MLTeamProject/blob/main/Random_Forest_and_LightGBM_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb

from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [25]:
def data_preprocessing(df):
    df.dropna()
    
    df = df.drop(df[df['예상'] == ':'].index)
    df = df.drop(df[df['도착'] == ':'].index)
    df = df.drop(df[df['현황'] == '취소'].index)

    df['연월일'] = df['연월일'].astype(str)
    df['년'] = df['연월일'].str[:4].astype(int)
    df['월'] = df['연월일'].str[5:7].astype(int)
    df['일'] = df['연월일'].str[8:10].astype(int)

    df['예상'] = df['예상'].astype(str)
    df['예상_1'] = df['예상'].str[:2].astype(int)
    df['예상_2'] = df['예상'].str[3:].astype(int)

    df['도착'] = df['도착'].astype(str)
    df['도착_1'] = df['도착'].str[:2].astype(int)
    df['도착_2'] = df['도착'].str[3:].astype(int)

    df = df[['년', '월', '일', '항공사', '편명', '출발지', '예상_1', '예상_2', '도착_1', '도착_2', '현황']]
    # for col in list(set(df.columns) - set(categorical)):
    #     df = df.astypes({col:'float64'})
    
    categorical = ['항공사', '편명', '출발지', '현황']
    label_encoder = LabelEncoder()
    for col in categorical:
        df[col] = label_encoder.fit_transform(df[col])
        df = df.astype({col:'int'})

    df.info()

    return df


In [29]:
data_train = pd.read_csv('./train_airportal.csv')
data_train = data_preprocessing(data_train)

data_test = pd.read_csv('./test_airportal.csv')
data_test = data_preprocessing(data_test)


x_train_tmp = data_train.iloc[:, :10]  # 0번부터 7번 열까지를 X로 설정
y_train_tmp = data_train.iloc[:, 10]   # 8번 열을 y로 설정

x_test_tmp = data_test.iloc[:, :10]  # 0번부터 7번 열까지를 X로 설정
y_test_tmp = data_test.iloc[:, 10]   # 8번 열을 y로 설정

print(x_test_tmp.info())
print(y_test_tmp.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487397 entries, 0 to 490035
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       487397 non-null  int64
 1   월       487397 non-null  int64
 2   일       487397 non-null  int64
 3   항공사     487397 non-null  int64
 4   편명      487397 non-null  int64
 5   출발지     487397 non-null  int64
 6   예상_1    487397 non-null  int64
 7   예상_2    487397 non-null  int64
 8   도착_1    487397 non-null  int64
 9   도착_2    487397 non-null  int64
 10  현황      487397 non-null  int64
dtypes: int64(11)
memory usage: 44.6 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 182019 entries, 0 to 184516
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       182019 non-null  int64
 1   월       182019 non-null  int64
 2   일       182019 non-null  int64
 3   항공사     182019 non-null  int64
 4   편명      182019 non-null  int64
 5   출발지     18201

In [30]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train_tmp, y_train_tmp)

y_pred = model.predict(x_test_tmp)
accuracy = accuracy_score(y_test_tmp, y_pred)

print("정확도:", accuracy)

정확도: 0.8218867261110104


In [33]:
def calculate_f1_score(y_true, y_pred):
    # 정밀도, 재현율, F1 점수 계산
    # micro, macro, samples, weighted
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    return f1

In [34]:
print(calculate_f1_score(y_test_tmp, y_pred))

0.8588758767310122


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train_tmp, y_train_tmp)

y_pred = model.predict(x_test_tmp)
accuracy = accuracy_score(y_test_tmp, y_pred)

print("정확도:", accuracy)

정확도: 0.9342705981243716


In [36]:
print(calculate_f1_score(y_test_tmp, y_pred))

0.9265630879197277


## Use airportal_passenger_airplane_per_day.csv

In [40]:
per_day_data = pd.read_csv('./per_day.csv')

In [43]:
per_day_data.info()
per_day_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487396 entries, 0 to 487395
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Unnamed: 0  487396 non-null  int64
 1   년           487396 non-null  int64
 2   월           487396 non-null  int64
 3   일           487396 non-null  int64
 4   항공사         487396 non-null  int64
 5   편명          487396 non-null  int64
 6   출발지         487396 non-null  int64
 7   예상_1        487396 non-null  int64
 8   예상_2        487396 non-null  int64
 9   도착_1        487396 non-null  int64
 10  도착_2        487396 non-null  int64
 11  현황          487396 non-null  int64
 12  예상_승객       487396 non-null  int64
 13  운행_편수       487396 non-null  int64
dtypes: int64(14)
memory usage: 52.1 MB


,Unnamed: 0,년,월,일,항공사,편명,출발지,예상_1,예상_2,도착_1,도착_2,현황,예상_승객,운행_편수
0,0,2016,1,1,49,441,9,0,5,0,11,1,1367,402
1,1,2016,1,1,70,1338,50,0,5,0,30,1,1367,402
2,2,2016,1,1,30,1440,184,0,10,0,15,1,1367,402
3,3,2016,1,1,75,1472,96,0,5,0,55,1,1367,402
4,4,2016,1,1,2,856,4,0,55,1,35,1,1367,402


In [62]:
x_tmp_1, x_tmp_2 = per_day_data.iloc[:, 1:11], per_day_data.iloc[:, 12:] # 독립 변수
y = per_day_data.iloc[:, 11] # 종속 변수
x = pd.concat([x_tmp_1, x_tmp_2], axis=1)

x.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487396 entries, 0 to 487395
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       487396 non-null  int64
 1   월       487396 non-null  int64
 2   일       487396 non-null  int64
 3   항공사     487396 non-null  int64
 4   편명      487396 non-null  int64
 5   출발지     487396 non-null  int64
 6   예상_1    487396 non-null  int64
 7   예상_2    487396 non-null  int64
 8   도착_1    487396 non-null  int64
 9   도착_2    487396 non-null  int64
 10  예상_승객   487396 non-null  int64
 11  운행_편수   487396 non-null  int64
dtypes: int64(12)
memory usage: 44.6 MB
<class 'pandas.core.series.Series'>
RangeIndex: 487396 entries, 0 to 487395
Series name: 현황
Non-Null Count   Dtype
--------------   -----
487396 non-null  int64
dtypes: int64(1)
memory usage: 3.7 MB


In [63]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [55]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9522979072630283


In [57]:
print(calculate_f1_score(y_test, y_pred))

0.9459081278519333


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
x_train.info()
x_test.info()
y_train.info()
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 389916 entries, 113211 to 121958
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   년       389916 non-null  int64
 1   월       389916 non-null  int64
 2   일       389916 non-null  int64
 3   항공사     389916 non-null  int64
 4   편명      389916 non-null  int64
 5   출발지     389916 non-null  int64
 6   예상_1    389916 non-null  int64
 7   예상_2    389916 non-null  int64
 8   도착_1    389916 non-null  int64
 9   도착_2    389916 non-null  int64
 10  예상_승객   389916 non-null  int64
 11  운행_편수   389916 non-null  int64
dtypes: int64(12)
memory usage: 38.7 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97480 entries, 448720 to 9253
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   년       97480 non-null  int64
 1   월       97480 non-null  int64
 2   일       97480 non-null  int64
 3   항공사     97480 non-null  int64
 4   편명      9748

In [65]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.933822322527698


In [67]:
print(calculate_f1_score(y_test, y_pred))

0.920480207661757


## Weather_add.csv

In [69]:
weather_add = pd.read_csv('./weather_add_16_18.csv')

In [70]:
weather_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485994 entries, 0 to 485993
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   년       485994 non-null  int64  
 1   월       485994 non-null  int64  
 2   일       485994 non-null  int64  
 3   항공사     485994 non-null  int64  
 4   편명      485994 non-null  int64  
 5   출발지     485994 non-null  int64  
 6   예상_1    485994 non-null  int64  
 7   예상_2    485994 non-null  int64  
 8   도착_1    485994 non-null  int64  
 9   도착_2    485994 non-null  int64  
 10  현황      485994 non-null  int64  
 11  예상_승객   485994 non-null  int64  
 12  운항편수    485994 non-null  int64  
 13  Count   485994 non-null  int64  
 14  가시거리    485994 non-null  float64
 15  온도      485994 non-null  float64
 16  풍속      485994 non-null  float64
dtypes: float64(3), int64(14)
memory usage: 63.0 MB


In [71]:
x_tmp_1, x_tmp_2 = per_day_data.iloc[:, :10], per_day_data.iloc[:, 11:] # 독립 변수
y = per_day_data.iloc[:, 10] # 종속 변수
x = pd.concat([x_tmp_1, x_tmp_2], axis=1)

x.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485994 entries, 0 to 485993
Data columns (total 16 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   년       485994 non-null  int64  
 1   월       485994 non-null  int64  
 2   일       485994 non-null  int64  
 3   항공사     485994 non-null  int64  
 4   편명      485994 non-null  int64  
 5   출발지     485994 non-null  int64  
 6   예상_1    485994 non-null  int64  
 7   예상_2    485994 non-null  int64  
 8   도착_1    485994 non-null  int64  
 9   도착_2    485994 non-null  int64  
 10  예상_승객   485994 non-null  int64  
 11  운항편수    485994 non-null  int64  
 12  Count   485994 non-null  int64  
 13  가시거리    485994 non-null  float64
 14  온도      485994 non-null  float64
 15  풍속      485994 non-null  float64
dtypes: float64(3), int64(13)
memory usage: 59.3 MB
<class 'pandas.core.series.Series'>
RangeIndex: 485994 entries, 0 to 485993
Series name: 현황
Non-Null Count   Dtype
--------------   -----
485994 non-null

In [72]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [73]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9456784534820317


In [74]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("정확도:", accuracy)

정확도: 0.9366145742240146


In [ ]:
lgb_classifier = lgb.LGBMClassifier(device='gpu')
param_grid = {
    'n_estimators': [20, 40, 60, 80, 100],
    'max_depth': [2, 5, 10, 15, 20],
    'num_leaves': [15, 31, 63, 127],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'min_child_samples': [20, 50, 100, 200],
}

# 그리드 서치를 학습 데이터에 적용하고 최적의 하이퍼파라미터를 찾기
grid_search = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print('Best parameters found by grid search:', grid_search.best_params_)

# 최적의 모델로 예측 및 정확도 측정
y_pred = grid_search.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy with best parameters:', accuracy)

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits


In [ ]:
from collections import product
from 

class TqdmGridSearchCV(GridSearchCV):
    """상속된 OS 그리드 서치 함수에 기반하여 작업 진행률 표시"""

    def _run_search(self, evaluate_candidates):
        """진행률 표시가능한 기능으로 Search 오버라이딩"""
        evaluate_candidates(ParameterGrid(self.param_grid))

    def _evaluate_candidates(self, candidate_params):
        out = []
        for parameters, (train, test) in product(candidate_params, enumerate(self.cv.split(X=self.X_, y=self.y_, groups=self.groups_))):
            score = self._calc_score(self.X_, self.y_, train=train, test=test, parameters=parameters)
            out.append((score, parameters, train, test))
        df = pd.DataFrame(out, columns=['{}_split_{}'.format(self.scoring, i) for i in range(5)]+['params'])
        self.cv_results_ = df
        return df

    def _calc_score(self, X, y, train, test, parameters):
        """모델 학습 및 테스트 점수 계산 """
        self.estimator.set_params(**parameters)
        self.estimator.fit(X[train], y[train], eval_set=(X[test], y[test]), verbose=False) # verbose=True to see training progress
        return self.estimator.score(X[test], y[test])


# 원래 코드에 진행률 표시 클래스 추가
from sklearn.utils import _safe_indexing
import numpy as np

lgb_classifier = lgb.LGBMClassifier(device='gpu')
param_grid = {
    'n_estimators': [20, 40, 60, 80, 100],
    'max_depth': [2, 5, 10, 15, 20],
    'num_leaves': [15, 31, 63, 127],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'min_child_samples': [20, 50, 100, 200],
}

grid_search = TqdmGridSearchCV(estimator=lgb_classifier, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(x_train, y_train)

print('Best parameters found by grid search:', grid_search.best_params_)

y_pred = grid_search.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy with best parameters:', accuracy)